In [11]:
#! pip install pytesseract opencv-python-headless mediapipe PyQt5


  Using cached pytesseract-0.3.13-py3-none-any.whl.metadata (11 kB)
  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)


ERROR: Could not find a version that satisfies the requirement mediapipe (from versions: none)
ERROR: No matching distribution found for mediapipe


In [12]:
#! pip install opencv-python mediapipe PyQt5



  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)


ERROR: Could not find a version that satisfies the requirement mediapipe (from versions: none)
ERROR: No matching distribution found for mediapipe


In [13]:
#! pip install PyQt5       


  Using cached PyQt5-5.15.11-cp38-abi3-win_amd64.whl.metadata (2.1 kB)
  Using cached PyQt5_Qt5-5.15.2-py3-none-win_amd64.whl.metadata (552 bytes)
Using cached PyQt5-5.15.11-cp38-abi3-win_amd64.whl (6.9 MB)
Using cached PyQt5_Qt5-5.15.2-py3-none-win_amd64.whl (50.1 MB)


In [14]:
#! pip install opencv-python-headless pytesseract mediapipe PyQt5


  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached pytesseract-0.3.13-py3-none-any.whl.metadata (11 kB)


ERROR: Could not find a version that satisfies the requirement mediapipe (from versions: none)
ERROR: No matching distribution found for mediapipe


In [16]:
#! pip install opencv-python

  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ----------------------- ---------------- 7.3/12.6 MB 43.3 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 42.8 MB/s eta 0:00:00


In [18]:
#! pip install pytesseract

  Using cached pytesseract-0.3.13-py3-none-any.whl.metadata (11 kB)
Using cached pytesseract-0.3.13-py3-none-any.whl (14 kB)
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 2.6/2.6 MB 27.9 MB/s eta 0:00:00


In [1]:
import sys
import cv2
import pytesseract
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QLabel, QFileDialog, QVBoxLayout, QWidget, QDialog
from PyQt5.QtGui import QPixmap, QImage
from PyQt5.QtCore import Qt

In [3]:
# Configuración de Tesseract (asegúrate de que la ruta sea correcta)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Diccionario de provincias y meses de matrícula
provincias = {
    "A": "Azuay", "B": "Bolívar", "C": "Carchi", "H": "Chimborazo", "X": "Cotopaxi",
    "E": "Esmeraldas", "J": "Galápagos", "G": "Guayas", "I": "Imbabura", "L": "Loja",
    "O": "Los Ríos", "M": "Manabí", "R": "Morona Santiago", "N": "Napo", "S": "Pastaza",
    "P": "Pichincha", "Q": "Orellana", "K": "Sucumbíos", "T": "Tungurahua", "U": "Zamora Chinchipe",
    "W": "Santa Elena", "Y": "Santo Domingo", "Z": "Zamora"
}

meses_matricula = {
    "1": "Febrero", "2": "Marzo", "3": "Abril", "4": "Mayo", "5": "Junio",
    "6": "Julio", "7": "Agosto", "8": "Septiembre", "9": "Octubre", "0": "Noviembre"
}

In [4]:
class ImageViewer(QDialog):
    def __init__(self, image, title="Imagen"):
        super().__init__()
        self.setWindowTitle(title)
        self.layout = QVBoxLayout()
        self.label = QLabel()
        self.layout.addWidget(self.label)
        self.setLayout(self.layout)
        
        # Convertir imagen OpenCV a formato Qt
        height, width, channel = image.shape
        bytes_per_line = 3 * width
        qimage = QImage(image.data, width, height, bytes_per_line, QImage.Format_RGB888).rgbSwapped()
        pixmap = QPixmap.fromImage(qimage)
        self.label.setPixmap(pixmap)
        self.resize(pixmap.size())


class App(QMainWindow):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('Reconocimiento de Placas Ecuatorianas')
        self.setGeometry(100, 100, 400, 300)

        layout = QVBoxLayout()

        label = QLabel("Seleccione una opción:")
        label.setAlignment(Qt.AlignCenter)
        layout.addWidget(label)

        btn_image = QPushButton("Reconocer placa en imagen", self)
        btn_image.clicked.connect(self.reconocer_placa_imagen)
        layout.addWidget(btn_image)

        btn_video = QPushButton("Reconocer placa en video", self)
        btn_video.clicked.connect(self.reconocer_placa_video)
        layout.addWidget(btn_video)

        btn_exit = QPushButton("Salir", self)
        btn_exit.clicked.connect(self.salir)
        layout.addWidget(btn_exit)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

    def extraer_datos_placa(self, texto_placa):
        texto_placa = texto_placa.strip().upper()
        if len(texto_placa) < 6:  # Validar longitud mínima de una placa
            return None, None

        provincia = provincias.get(texto_placa[0], "Desconocida")
        ultimo_digito = texto_placa[-1]
        mes = meses_matricula.get(ultimo_digito, "Desconocido")
        return provincia, mes

    def mostrar_info_placa(self, image, provincia, mes):
        cv2.putText(image, f"Provincia: {provincia}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(image, f"Mes Matricula: {mes}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    def reconocer_placa_imagen(self):
        options = QFileDialog.Options()
        file_path, _ = QFileDialog.getOpenFileName(self, "Seleccionar Imagen", "", "Archivos de Imagen (*.png *.jpg *.jpeg)", options=options)

        if not file_path:
            print("No se seleccionó ningún archivo.")
            return

        image = cv2.imread(file_path)
        if image is None:
            print("Error al leer la imagen.")
            return

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        texto_placa = pytesseract.image_to_string(gray, config='--oem 3 --psm 8').strip()
        print(f"Texto detectado: {texto_placa}")

        provincia, mes = self.extraer_datos_placa(texto_placa)
        if provincia and mes:
            self.mostrar_info_placa(image, provincia, mes)
        else:
            cv2.putText(image, "Placa no detectada o inválida", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Mostrar imagen en el visor de PyQt5
        viewer = ImageViewer(image, title="Reconocimiento de Placa")
        viewer.exec_()

    def reconocer_placa_video(self):
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print("Error al abrir la cámara.")
            return

        while True:
            ret, frame = cap.read()
            if not ret:
                print("Error al leer el frame del video.")
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            texto_placa = pytesseract.image_to_string(gray, config='--oem 3 --psm 8').strip()
            provincia, mes = self.extraer_datos_placa(texto_placa)

            if provincia and mes:
                self.mostrar_info_placa(frame, provincia, mes)
            else:
                cv2.putText(frame, "Placa no detectada", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            cv2.imshow("Reconocimiento de Placa en Video", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

    def salir(self):
        self.close()


if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = App()
    ex.show()
    sys.exit(app.exec_())


c:\Users\gabri\OneDrive\Escritorio\ProyectoInteligenciaArtificial\EntornoVirt\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  """Entry point for launching an IPython kernel.
c:\Users\gabri\OneDrive\Escritorio\ProyectoInteligenciaArtificial\EntornoVirt\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead


No se seleccionó ningún archivo.


SystemExit: 0

c:\Users\gabri\OneDrive\Escritorio\ProyectoInteligenciaArtificial\EntornoVirt\lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
